In [4]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 

In [5]:
df = pd.read_csv("SuperStore (1).csv")
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,08-11-2017,11-11-2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,08-11-2017,11-11-2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,12-06-2017,16-06-2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,11-10-2016,18-10-2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11-10-2016,18-10-2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680


In [6]:
df = df.drop_duplicates()

In [7]:
df.isnull().sum()

Row ID            0
Order ID          0
Order Date        0
Ship Date         0
Ship Mode         0
Customer ID       0
Customer Name     0
Segment           0
Country           0
City              0
State             0
Postal Code      11
Region            0
Product ID        0
Category          0
Sub-Category      0
Product Name      0
Sales             0
dtype: int64

In [8]:
df['Postal Code'].dropna().skew()

-0.13129452199239716

In [9]:
df['Postal Code'].dtype

dtype('float64')

In [10]:
mode_pc = df['Postal Code'].mode()[0]
df['Postal Code'] = df['Postal Code'].fillna(mode_pc)

In [11]:
df['Postal Code'] = df['Postal Code'].astype('Int64')

In [12]:
df['Order Date'] = pd.to_datetime(df['Order Date'], dayfirst=True)
df['Ship Date'] = pd.to_datetime(df['Ship Date'], dayfirst=True)

In [13]:
df['Category'].unique()
df['Sub-Category'].unique()
df['Region'].unique()

array(['South', 'West', 'Central', 'East'], dtype=object)

In [14]:
df.columns

Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State',
       'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category',
       'Product Name', 'Sales'],
      dtype='object')

In [15]:
df['Estimated Profit'] = df['Sales'] * 0.25

In [16]:
product_df = df.groupby('Product Name').agg({
    'Sales': 'sum',
    'Estimated Profit': 'sum',
    'Category': 'first',
    'Sub-Category': 'first',
}).reset_index()

In [17]:
product_df = product_df[product_df['Estimated Profit'] > 0].reset_index(drop=True)

In [18]:
product_df = product_df.sort_values(by='Estimated Profit', ascending=False).reset_index(drop=True)

In [20]:
pip install pulp

   ---------------------------------------- 0.0/16.4 MB ? eta -:--:--
   - -------------------------------------- 0.5/16.4 MB 5.6 MB/s eta 0:00:03
   --- ------------------------------------ 1.3/16.4 MB 4.5 MB/s eta 0:00:04
   ----- ---------------------------------- 2.1/16.4 MB 3.8 MB/s eta 0:00:04
   ------- -------------------------------- 2.9/16.4 MB 3.7 MB/s eta 0:00:04
   -------- ------------------------------- 3.4/16.4 MB 3.7 MB/s eta 0:00:04
   --------- ------------------------------ 3.9/16.4 MB 3.5 MB/s eta 0:00:04
   ---------- ----------------------------- 4.5/16.4 MB 3.2 MB/s eta 0:00:04
   ------------ --------------------------- 5.0/16.4 MB 3.2 MB/s eta 0:00:04
   -------------- ------------------------- 5.8/16.4 MB 3.1 MB/s eta 0:00:04
   --------------- ------------------------ 6.3/16.4 MB 3.1 MB/s eta 0:00:04
   ----------------- ---------------------- 7.1/16.4 MB 3.2 MB/s eta 0:00:03
   ------------------ --------------------- 7.6/16.4 MB 3.1 MB/s eta 0:00:03
   ---

In [21]:
from pulp import LpProblem, LpVariable, LpBinary, lpSum, LpMaximize, value

In [22]:
prob = LpProblem("Maximize_Profit", LpMaximize)

In [23]:
product_vars = {
    row['Product Name']: LpVariable(f"x_{i}", cat=LpBinary)
    for i, row in product_df.iterrows()
}

In [24]:
prob += lpSum([
    product_vars[row['Product Name']] * row['Estimated Profit']
    for _, row in product_df.iterrows()
])

In [25]:
BUDGET = 5000
prob += lpSum([
    product_vars[row['Product Name']] * row['Sales']
    for _, row in product_df.iterrows()
]) <= BUDGET

In [26]:
MAX_PRODUCTS = 20
prob += lpSum(product_vars.values()) <= MAX_PRODUCTS

In [27]:
prob.solve()

1

In [28]:
selected = []

In [29]:
for i, row in product_df.iterrows():
    if product_vars[row['Product Name']].varValue == 1:
        selected.append(row)

selected_df = pd.DataFrame(selected)

In [34]:
selected_df

,Product Name,Sales,Estimated Profit,Category,Sub-Category
872,I.R.I.S IRISCard Anywhere 5 Card Scanner,350.973,87.74325,Technology,Machines
966,Plastic Binding Combs,269.670,67.41750,Office Supplies,Binders
971,Eldon Wave Desk Accessories,266.730,66.68250,Furniture,Furnishings
972,Avery 475,266.400,66.60000,Office Supplies,Labels
973,Belkin F9S820V06 8 Outlet Surge,266.336,66.58400,Office Supplies,Appliances
974,Belkin Standard 104 key USB Keyboard,265.538,66.38450,Technology,Accessories
975,"Boston Model 1800 Electric Pencil Sharpener, Gray",264.610,66.15250,Office Supplies,Art
978,Staple remover,263.088,65.77200,Office Supplies,Supplies
979,"Southworth 100% Résumé Paper, 24lb.",262.964,65.74100,Office Supplies,Paper
980,Acco Data Flex Cable Posts For Top & Bottom Lo...,262.836,65.70900,Office Supplies,Binders


In [35]:
sorted_df = product_df.sort_values(by='Estimated Profit', ascending=False).reset_index(drop=True)
sorted_df['Cumulative Profit'] = sorted_df['Estimated Profit'].cumsum()
total_profit = sorted_df['Estimated Profit'].sum()
sorted_df['Cumulative Profit %'] = 100 * sorted_df['Cumulative Profit'] / total_profit

In [40]:
top_cutoff = sorted_df[sorted_df['Cumulative Profit %'] <= 75]
top_percent = 100 * len(top_cutoff) / len(product_df)

print(f"{top_percent:.1f}% of products contribute to 75% of estimated profit")

18.5% of products contribute to 75% of estimated profit


In [41]:
product_df['Profit Efficiency'] = product_df['Estimated Profit'] / product_df['Sales']
category_efficiency = product_df.groupby('Category')['Profit Efficiency'].mean().sort_values()

print(category_efficiency)


Category
Furniture          0.25
Office Supplies    0.25
Technology         0.25
Name: Profit Efficiency, dtype: float64
